In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import numpy as np

In [3]:
(x_train, y_train), (x_val, y_val) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=2, validation_data=(x_val, y_val), batch_size=32)

for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(x_train, y_train, epochs=2, validation_data=(x_val, y_val), batch_size=32)

model.save('fine_tuned_vgg16_cifar10.h5')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 0.2566 - loss: 2.0847 - val_accuracy: 0.4834 - val_loss: 1.5201
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.4614 - loss: 1.5515 - val_accuracy: 0.5204 - val_loss: 1.3935
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 29s 16ms/step - accuracy: 0.5574 - loss: 1.2683 - val_accuracy: 0.6633 - val_loss: 0.9715
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - accuracy: 0.6646 - loss: 0.9622 - val_accuracy: 0.6947 - val_loss: 0.8731


In [11]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

print (acc,val_acc, loss, val_loss)


[0.3446600139141083, 0.4731599986553192] [0.48339998722076416, 0.5203999876976013] [1.8471741676330566, 1.516546368598938] [1.5201051235198975, 1.3934643268585205]
